In [1]:
import requests
import pandas as pd

# ==================================================
# 1. DESCARGA DATOS SEC
# ==================================================
def fetch_company_facts(cik: str) -> dict:
    url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    headers = {
        "User-Agent": "Academic research (tuemail@dominio.com)"
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()


# ==================================================
# 2. CONSTRUCTOR GENERICO POR UNIDAD XBRL
# ==================================================
def build_unit_dataframe(data: dict, cik: str, unit: str) -> pd.DataFrame:
    records = []

    for account, account_data in data.get("facts", {}).get("us-gaap", {}).items():
        units = account_data.get("units", {})
        if unit not in units:
            continue

        for item in units[unit]:
            records.append({
                "cik": cik,
                "account": account,
                "start": item.get("start"),
                "end": item.get("end"),
                "value": item.get("val"),
                "form": item.get("form"),
                "fy": item.get("fy"),
                "fp": item.get("fp")
            })

    df = pd.DataFrame(records)

    df["start"] = pd.to_datetime(df["start"], errors="coerce")
    df["end"] = pd.to_datetime(df["end"], errors="coerce")
    df["fy"] = pd.to_numeric(df["fy"], errors="coerce")
    df["duracion_dias"] = (df["end"] - df["start"]).dt.days

    return df


# ==================================================
# 3. INCOME STATEMENT
# ==================================================
def get_income_statement(df: pd.DataFrame, years: list[int]) -> pd.DataFrame:
    accounts = [
        "SalesRevenueNet",
        "RevenueFromContractWithCustomerExcludingAssessedTax",
        "CostOfGoodsAndServicesSold",
        "GrossProfit",
        "OperatingExpenses",
        "ResearchAndDevelopmentExpense",
        "SellingGeneralAndAdministrativeExpense",
        "OperatingIncomeLoss",
        "NonoperatingIncomeExpense",
        "InterestExpense",
        "IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest",
        "IncomeTaxExpenseBenefit",
        "NetIncomeLoss"
    ]

    df_is = df[
        (df["form"] == "10-K") &
        (df["fp"] == "FY") &
        (df["duracion_dias"] > 300) &
        (df["fy"].isin(years)) &
        (df["account"].isin(accounts))
    ].copy()

    df_is = df_is.drop(columns=["cik", "start", "form", "fy", "fp", "duracion_dias"])

    return df_is.pivot_table(
        index="end",
        columns="account",
        values="value",
        aggfunc="last"
    ).sort_index()


# ==================================================
# 4. BALANCE SHEET
# ==================================================
def get_balance_sheet(df: pd.DataFrame, years: list[int]) -> pd.DataFrame:
    accounts = [
        "Assets", "AssetsCurrent", "AssetsNoncurrent",
        "CashAndCashEquivalentsAtCarryingValue",
        "MarketableSecuritiesCurrent",
        "MarketableSecuritiesNoncurrent",
        "AccountsReceivableNetCurrent",
        "InventoryNet",
        "PropertyPlantAndEquipmentNet",
        "OtherAssetsCurrent", "OtherAssetsNoncurrent",
        "Liabilities", "LiabilitiesCurrent", "LiabilitiesNoncurrent",
        "AccountsPayableCurrent", "OtherLiabilitiesCurrent",
        "OtherLiabilitiesNoncurrent",
        "LongTermDebt", "LongTermDebtCurrent", "LongTermDebtNoncurrent",
        "CommercialPaper",
        "StockholdersEquity",
        "CommonStocksIncludingAdditionalPaidInCapital",
        "RetainedEarningsAccumulatedDeficit",
        "AccumulatedOtherComprehensiveIncomeLossNetOfTax",
        "LiabilitiesAndStockholdersEquity"
    ]

    df_bs = df[
        (df["form"] == "10-K") &
        (df["fp"] == "FY") &
        (df["start"].isna()) &
        (df["fy"].isin(years)) &
        (df["account"].isin(accounts))
    ].copy()

    df_bs = df_bs.drop(columns=["cik", "start", "form", "fy", "fp", "duracion_dias"])

    return df_bs.pivot_table(
        index="end",
        columns="account",
        values="value",
        aggfunc="last"
    ).sort_index()


# ==================================================
# 5. CASH FLOW STATEMENT
# ==================================================
def get_cash_flow_statement(df: pd.DataFrame, years: list[int]) -> pd.DataFrame:
    accounts = [
        "CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents",
        "NetIncomeLoss",
        "DepreciationDepletionAndAmortization",
        "ShareBasedCompensation",
        "OtherNoncashIncomeExpense",
        "IncreaseDecreaseInAccountsReceivable",
        "IncreaseDecreaseInOtherReceivables",
        "IncreaseDecreaseInInventories",
        "IncreaseDecreaseInOtherOperatingAssets",
        "IncreaseDecreaseInAccountsPayable",
        "IncreaseDecreaseInOtherOperatingLiabilities",
        "NetCashProvidedByUsedInOperatingActivities",
        "PaymentsToAcquireAvailableForSaleSecuritiesDebt",
        "ProceedsFromMaturitiesPrepaymentsAndCallsOfAvailableForSaleSecurities",
        "ProceedsFromSaleOfAvailableForSaleSecuritiesDebt",
        "PaymentsToAcquirePropertyPlantAndEquipment",
        "PaymentsForProceedsFromOtherInvestingActivities",
        "NetCashProvidedByUsedInInvestingActivities",
        "PaymentsRelatedToTaxWithholdingForShareBasedCompensation",
        "PaymentsOfDividends",
        "PaymentsForRepurchaseOfCommonStock",
        "ProceedsFromIssuanceOfLongTermDebt",
        "RepaymentsOfLongTermDebt",
        "ProceedsFromRepaymentsOfCommercialPaper",
        "ProceedsFromPaymentsForOtherFinancingActivities",
        "NetCashProvidedByUsedInFinancingActivities",
        "CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect",
        "IncomeTaxesPaidNet"
    ]

    df_cf = df[
        (df["form"] == "10-K") &
        (df["fp"] == "FY") &
        (df["fy"].isin(years)) &
        (df["end"].dt.month == 9) &
        ((df["start"].dt.month.isin([9, 10])) | (df["start"].isna())) &
        (df["account"].isin(accounts))
    ].copy()

    df_cf = df_cf.drop(columns=["cik", "start", "form", "fy", "fp", "duracion_dias"])

    return df_cf.pivot_table(
        index="end",
        columns="account",
        values="value",
        aggfunc="last"
    ).sort_index()


# ==================================================
# 6. SHARES / CAPITAL STRUCTURE
# ==================================================
def get_shares_statement(df: pd.DataFrame, years: list[int]) -> pd.DataFrame:
    accounts = [
        "WeightedAverageNumberOfSharesOutstandingBasic",
        "WeightedAverageNumberOfDilutedSharesOutstanding",
        "CommonStockSharesOutstanding",
        "CommonStockSharesIssued"
    ]

    df_sh = df[
        (df["form"] == "10-K") &
        (df["fp"] == "FY") &
        (df["fy"].isin(years)) &
        (df["end"].dt.month == 9) &
        ((df["start"].dt.month.isin([9, 10])) | (df["start"].isna())) &
        (df["account"].isin(accounts))
    ].copy()

    df_sh = df_sh.drop(columns=["cik", "start", "form", "fy", "fp", "duracion_dias"])

    return df_sh.pivot_table(
        index="end",
        columns="account",
        values="value",
        aggfunc="last"
    ).sort_index()


# ==================================================
# 7. PIPELINE COMPLETO
# ==================================================
def get_financial_statements(cik: str,
                             is_years: list[int],
                             bs_years: list[int],
                             cf_years: list[int],
                             sh_years: list[int]):

    data = fetch_company_facts(cik)

    df_usd = build_unit_dataframe(data, cik, unit="USD")
    df_shares = build_unit_dataframe(data, cik, unit="shares")

    df_is = get_income_statement(df_usd, is_years)
    df_bs = get_balance_sheet(df_usd, bs_years)
    df_cf = get_cash_flow_statement(df_usd, cf_years)
    df_sh = get_shares_statement(df_shares, sh_years)

    df_all = pd.concat(
        {
            "IncomeStatement": df_is,
            "BalanceSheet": df_bs,
            "CashFlowStatement": df_cf,
            "Shares": df_sh
        },
        axis=1
    )

    return df_is, df_bs, df_cf, df_sh, df_all


In [2]:
cik = "0000320193"

df_is, df_bs, df_cf, df_sh, df_all = get_financial_statements(
    cik=cik,
    is_years=[2016, 2019, 2022, 2025],
    bs_years=[2017, 2019, 2021, 2023, 2025],
    cf_years=[2016, 2019, 2022, 2025],
    sh_years=[2016, 2019, 2022, 2025]
)

df_all.to_excel("APPLE_AllStatements.xlsx")
df_cf

account,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect,DepreciationDepletionAndAmortization,IncomeTaxesPaidNet,IncreaseDecreaseInAccountsPayable,IncreaseDecreaseInAccountsReceivable,IncreaseDecreaseInInventories,IncreaseDecreaseInOtherOperatingAssets,IncreaseDecreaseInOtherOperatingLiabilities,IncreaseDecreaseInOtherReceivables,...,PaymentsRelatedToTaxWithholdingForShareBasedCompensation,PaymentsToAcquireAvailableForSaleSecuritiesDebt,PaymentsToAcquirePropertyPlantAndEquipment,ProceedsFromIssuanceOfLongTermDebt,ProceedsFromMaturitiesPrepaymentsAndCallsOfAvailableForSaleSecurities,ProceedsFromPaymentsForOtherFinancingActivities,ProceedsFromRepaymentsOfCommercialPaper,ProceedsFromSaleOfAvailableForSaleSecuritiesDebt,RepaymentsOfLongTermDebt,ShareBasedCompensation
end,,,,,,,,,,,,,,,,,,,,,
2014-09-27,NaN,NaN,NaN,1.002600e+10,5.938000e+09,4.232000e+09,7.600000e+07,-1.670000e+08,6.010000e+09,2.220000e+09,...,1.158000e+09,NaN,9.571000e+09,1.196000e+10,1.881000e+10,NaN,6.306000e+09,NaN,0.000000e+00,2.863000e+09
2015-09-26,NaN,NaN,NaN,1.325200e+10,5.400000e+09,-6.110000e+08,2.380000e+08,1.790000e+08,8.746000e+09,3.735000e+09,...,1.499000e+09,NaN,1.124700e+10,2.711400e+10,1.453800e+10,NaN,2.191000e+09,NaN,0.000000e+00,3.586000e+09
2016-09-24,2.048400e+10,NaN,NaN,1.044400e+10,1.791000e+09,-1.095000e+09,-2.170000e+08,-1.090000e+09,-2.104000e+09,5.100000e+07,...,1.570000e+09,NaN,1.273400e+10,2.495400e+10,2.125800e+10,NaN,-3.970000e+08,NaN,2.500000e+09,4.210000e+09
2017-09-30,2.028900e+10,-1.950000e+08,1.015700e+10,1.159100e+10,8.966000e+09,2.093000e+09,2.723000e+09,5.318000e+09,1.092000e+09,4.254000e+09,...,1.874000e+09,1.594860e+11,1.245100e+10,2.866200e+10,3.177500e+10,0.0,3.852000e+09,9.456400e+10,3.500000e+09,4.840000e+09
2018-09-29,2.591300e+10,5.624000e+09,1.090300e+10,1.041700e+10,9.175000e+09,5.322000e+09,-8.280000e+08,4.230000e+08,3.844900e+10,8.010000e+09,...,2.527000e+09,7.135600e+10,1.331300e+10,6.969000e+09,5.588100e+10,0.0,-3.700000e+07,4.783800e+10,6.500000e+09,5.340000e+09
2019-09-28,5.022400e+10,2.431100e+10,1.254700e+10,1.526300e+10,-1.923000e+09,-2.450000e+08,2.890000e+08,-8.730000e+08,-4.700000e+09,-2.931000e+09,...,2.817000e+09,3.963000e+10,1.049500e+10,6.963000e+09,4.010200e+10,-105000000.0,-5.977000e+09,5.698800e+10,8.805000e+09,6.068000e+09
2020-09-26,3.978900e+10,-1.043500e+10,1.105600e+10,9.501000e+09,-4.062000e+09,-6.917000e+09,1.270000e+08,9.588000e+09,8.916000e+09,-1.553000e+09,...,3.634000e+09,1.149380e+11,7.309000e+09,1.609100e+10,6.991800e+10,754000000.0,-9.630000e+08,5.047300e+10,1.262900e+10,6.829000e+09
2021-09-25,3.592900e+10,-3.860000e+09,1.128400e+10,2.538500e+10,1.232600e+10,1.012500e+10,2.642000e+09,8.042000e+09,5.799000e+09,3.903000e+09,...,6.556000e+09,1.095580e+11,1.108500e+10,2.039300e+10,5.902300e+10,976000000.0,1.022000e+09,4.746000e+10,8.750000e+09,7.906000e+09
2022-09-24,2.497700e+10,-1.095200e+10,1.110400e+10,1.957300e+10,9.448000e+09,1.823000e+09,-1.484000e+09,6.499000e+09,5.632000e+09,7.520000e+09,...,6.223000e+09,7.692300e+10,1.070800e+10,5.465000e+09,2.991700e+10,-160000000.0,3.955000e+09,3.744600e+10,9.543000e+09,9.038000e+09
